# Segmenting and Clustering Neighborhoods in Toronto
## by Dalia Y. Domínguez 

In [1]:
#Import the libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

Use the requests library to download the webpage https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.
Save the text of the response as a variable named html_data and make the object beautiful_soup.

In [45]:
# url and get ready the information we'll need 
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data  = requests.get(url).text 
beautiful_soup= BeautifulSoup(html_data, 'html.parser')

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [49]:
# get ready the data frame
torontoPC = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"]) #headers
torontoPC

,PostalCode,Borough,Neighborhood


We're only process the cells that have an assigned borough.
if more than one neighborhood can exist in one postal code area. It will will have in one row the neighborhoods separated with a comma as shown.
If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [ ]:
for row in beautiful_soup.table.find_all("tr"):
    for postalCodeInf in row.find_all("td"):
        if ( postalCodeInf.span.text != 'Not assigned'):
            postalCode = postalCodeInf.b.text
            borough = postalCodeInf.span.text.split('(')[0]
            neighborhood = postalCodeInf.span.text.split('(')[1].replace('/',',')[:-1] 
            torontoPC = torontoPC.append({"PostalCode":postalCode, "Borough":borough, "Neighborhood":neighborhood}, ignore_index=True)
            
torontoPC['Borough']=torontoPC ['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

#torontoPC[torontoPC['Neighborhood']=='Not assigned'] #verified if there's a missing neighborhood

Data Frame:

In [61]:
torontoPC

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


Number of rows of the frame

In [59]:
torontoPC.shape

(103, 3)